# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [6]:
# Load environment variables
%load_ext dotenv
%dotenv 
# Add src to path
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Standard libraries
import pandas as pd
import numpy as np

cannot find .env file


In [7]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\UFT data science program\DSI_1\scaling_to_production\05_src\data\adult\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target (Y)
X = adult_dt.drop(columns='income')  # All columns except 'income'
Y = adult_dt['income']  # The 'income' column

# Split the data into training and testing sets with a 70-30% ratio
# Setting random_state=42 for reproducibility
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shape of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


In [12]:
df_raw = pd.read_csv(r'C:\UFT data science program\DSI_1\scaling_to_production\05_src\data\adult\adult.data', header = None, names = columns)

In [13]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

What is the random state of the splitting function?
The random_state parameter in the train_test_split function is used to control the shuffling of the data before splitting it into training and testing sets. Setting a specific random_state ensures that the split is reproducible, meaning you get the same split every time you run the code with that specific random_state value.

Why is it useful?
Setting the random_state parameter in data splitting functions like train_test_split enhances reproducibility, stability, and fairness in model comparison and evaluation. It ensures that the randomness in your experiments is controlled and consistent, making your work more robust, reliable, and easier to validate.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
import pandas as pd

# Sample data frame to identify numerical and categorical columns
# Replace this with your actual dataframe
df = pd.DataFrame({
    'numerical1': [1.0, 2.0, None, 4.0],
    'numerical2': [10, 20, None, 40],
    'categorical1': ['A', 'B', None, 'A'],
    'categorical2': ['X', None, 'Y', 'Y']
})

# Identify numerical and categorical columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns

# Define the transformer for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the transformer for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ]
)
preprocessor
# Example of how to apply this preprocessor to a dataset
# transformed_df = preprocessor.fit_transform(df)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 Index(['numerical1', 'numerical2'], dtype='object')),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 Index(['categorical1', 'categorical2'], dtype='object'))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [45]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Step 1: Load your actual DataFrame
file_path = r'C:\UFT data science program\DSI_1\scaling_to_production\05_src\data\adult\adult.data'
df = pd.read_csv(file_path)

# Step 2: Identify numerical and categorical columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns

# Define the transformer for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the transformer for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])


# Step 5: Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ]
)

# Step 6: Create a pipeline with preprocessing and classifier
pipe_simple = Pipeline([
    ('preprocess', preprocessor),
    ('model', RandomForestClassifier())
])

# Display the pipeline
pipe_simple


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['39', ' 77516', ' 13', ' 2174', ' 0', ' 40'], dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  Index([' State-gov', ' Bachelors', ' Never-married', ' Adm-clerical',
       ' Not-in-family', ' White', ' Male', ' United-States', ' <=50K'],
      dtype='object'))])),
                ('model', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [53]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB

In [67]:

#  Define the scoring metrics
scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

#  Perform cross-validation on the training set
res_simple_dict = cross_validate(
    pipe_simple,
    X_train,
    Y_train,
    cv=5,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1
)

#  Convert the results dictionary to a DataFrame and add an 'experiment' column
res_simple = pd.DataFrame(res_simple_dict).assign(experiment=1)

# Display the cross-validation results
print(res_simple)

#  Evaluate the fitted model on the test set
pipe_simple.fit(X_train, Y_train)
test_score = pipe_simple.score(X_test, Y_test)
print(f"\nTest Set Accuracy: {test_score:.4f}")

# Detailed evaluation on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

Y_pred = pipe_simple.predict(X_test)
Y_proba = pipe_simple.predict_proba(X_test)[:, 1]  # For ROC AUC and log loss

print("\nTest Set Detailed Metrics:")
print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.4f}")
print(f"Precision: {precision_score(Y_test, Y_pred):.4f}")
print(f"Recall: {recall_score(Y_test, Y_pred):.4f}")
print(f"F1 Score: {f1_score(Y_test, Y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(Y_test, Y_proba):.4f}")
print(f"Log Loss: {log_loss(Y_test, Y_proba):.4f}")


    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  12.151813    0.227012          -0.379297           -0.080600      0.893879   
1  12.482668    0.238467          -0.346505           -0.081736      0.904273   
2  12.457520    0.255141          -0.372077           -0.081460      0.902922   
3  12.464814    0.238467          -0.377147           -0.082766      0.905758   
4  12.444925    0.230608          -0.362360           -0.082112      0.904750   

   train_roc_auc   test_f1  train_f1  test_accuracy  train_accuracy  \
0            1.0  0.646394       1.0       0.843762             1.0   
1            1.0  0.672559       1.0       0.850672             1.0   
2            1.0  0.653328       1.0       0.847025             1.0   
3            1.0  0.677615       1.0       0.856786             1.0   
4            1.0  0.684188       1.0       0.859282             1.0   

   test_precision  train_precision  test_recall  train_recall  experiment  
0        0

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [69]:

# Add fold number to the DataFrame for clarity
res_simple['fold'] = range(1, len(res_simple) + 1)

#  Sort the DataFrame by the negative log loss of the test set
sorted_res_simple = res_simple.sort_values(by='test_neg_log_loss', ascending=False)

#  Display the sorted DataFrame
print("Cross-Validation Results Sorted by Negative Log Loss (Validation Set):")
print(sorted_res_simple[['fold', 'test_neg_log_loss', 'test_roc_auc', 'test_f1', 
                         'test_accuracy', 'test_precision', 'test_recall']])


Cross-Validation Results Sorted by Negative Log Loss (Validation Set):
   fold  test_neg_log_loss  test_roc_auc   test_f1  test_accuracy  \
1     2          -0.346505      0.904273  0.672559       0.850672   
4     5          -0.362360      0.904750  0.684188       0.859282   
2     3          -0.372077      0.902922  0.653328       0.847025   
3     4          -0.377147      0.905758  0.677615       0.856786   
0     1          -0.379297      0.893879  0.646394       0.843762   

   test_precision  test_recall  
1        0.712121     0.637161  
4        0.744142     0.633174  
2        0.718660     0.598884  
3        0.739623     0.625199  
0        0.709924     0.593301  


Calculate the mean of each metric. 

In [71]:

# Calculate the mean of each metric
mean_metrics = res_simple.mean()

# Display the mean metrics
print("Mean Cross-Validation Metrics:")
print(mean_metrics)


Mean Cross-Validation Metrics:
fit_time              12.343289
score_time             0.238833
test_neg_log_loss     -0.367477
train_neg_log_loss    -0.081735
test_roc_auc           0.902316
train_roc_auc          1.000000
test_f1                0.666817
train_f1               1.000000
test_accuracy          0.851505
train_accuracy         1.000000
test_precision         0.724894
train_precision        1.000000
test_recall            0.617544
train_recall           1.000000
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [74]:


#  Predict probabilities and class labels on the testing data
Y_pred_proba = pipe_simple.predict_proba(X_test)  # Probability predictions
Y_pred = pipe_simple.predict(X_test)              # Class predictions

#  Calculate performance metrics
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),             # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),        # ROC AUC score
    'accuracy': accuracy_score(Y_test, Y_pred),                  # Accuracy
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred) # Balanced accuracy
}

# Display the calculated metrics
print("Test Set Performance Metrics:")
print(test_metrics)


Test Set Performance Metrics:
{'neg_log_loss': -0.3857076055170705, 'roc_auc': 0.9039583548539247, 'accuracy': 0.8585905112851221, 'balanced_accuracy': 0.7830749488464662}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.